# eTADA
A statistical framework for mapping risk genes from *de novo* mutations in whole-genome sequencing studies

## Introduction
With the fast pace of technology revolution in the field of genomics, whole-exome sequencing (WES) and whole-genome sequencing (WGS) have become more and more affordable. We have seen rapid accumulation of these sequencing data from cohorts with different traits and diseases and are about to see even more in the near future. Now, the major challenge is what we could learn from these data. Specifically, **could we use mutation data to learn what types of mutations are relevant to disease eitiology and which genes are likely to be risk genes**. TADA-A is a statistical framework designed to answer these important questions using *de novo **SNV** mutations* (DNM), one type of mutations that spontaneously arise in an offspring and not present in its parents. The two main types of input data for TADA-A is DNM and functional/conservational annotation data. It works by integrating DNM information across different studies while accounting for technical variations that might affect observed DNM mutation rates among these studies. It first adjusts baseline mutation rates for each study and estimates the effect sizes of annotations using DNM data from all studies. Finally, it uses relevant annotations to predict the deleteriouss of each mutation and predict disease risk genes. 



eTADA is developed on the foundation of TADA-A. It estimates the mutation rates of study-specific *de novo* frameshift INDELs and integrated the weights of both SNVs and frameshift INDELs to increase the statistical power for risk gene discovery. Besides, its speed of parameter estimation is approximately three times faster than TADA-A's. This tool is freely available at https://github.com/yfu1116/eTADA. 


In [ ]:
library(eTADA)
options(scipen=20)

# adjust mutation rates

--mut_file

A string representing a DNM bed file with three columns, separated by "\t". Notice, here the mutation file does not need to have allele information.
chr \t start(0-based) \t end(1-based)

In [6]:
head -n 2 mut_file

chr12	123187419	123187420
chr8	22550355	22550356


--window_file

A window file that has several columns. Each row is a 50-bp window with coordinates information, uncalibrated baseline mutation rates, its associated gene name and other features that are used to adjust baseline mutation rates. A example window file is provided as `../MS_data/Example_windows_with_div_score.bed` and the first four rows are shown below. If you use your own window file,  the first 6 columns are required and the names of these columns should not be changed.  The rest of the columns are features that are used to adjust mutation rates so could be customized. 

In [8]:
head(f,2)

chr,start,end,site_index,genename,mutrate,coding,promoter,GC_content,div_score
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>
chr1,68090,68140,1,OR4F5,7.29662e-07,0,1,0.38,0.07401378
chr1,68140,68190,2,OR4F5,5.25763e-07,0,1,0.38,0.07401378


--sample_size

A integer representing the number of individuals in the study

--scale_features

A vector showing the names of the features for which normalization are needed when adjusting mutation rates. 

--scaling_file_name

A string giving the name of the output file that gives a mutation rate scaling factor for each window, which will be used in the following steps. The first three rows of an example output file is shown below. As you can see, we use `site_index` as the main identifier to link the window file and the mutation rate scaling file. 

In [11]:
head -n 3 scaling_file_name

site_index	scaling_factor
1	0.948531063057671
5	0.768557586845165


In [4]:
TADA_A_adjust_mutrate(
mut_file="/storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/no_overlp_wd/all.17428.no_overlp_wd.vep.syn.6788.bed",
window_file="/storage11_7T/fuy/TADA-A/db/MS_data/windows_partition/no_overlp.17428.gene.syn.new_mutrate.bed",
sample_size=6430,
scale_features=c("GC_content"),
scaling_file_name="/storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/t.txt",
mutrate_mode = "regular",
genes = "all"
)

[1] "Made a tmp folder for tmp files."
[1] "Counted mutation over windows. Time consumed: 1.526s"
[1] "Chose mutrate mode: regular."
[1] "Finished fiting mutation rate model and calculating scaling factors. Time consumed: 3.188s"
[1] "Removed temporary files."


,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-0.1396165,0.02494531,-5.596903,0.00000002182153
GC_content,-0.1329139,0.02360769,-5.630111,0.00000001800937


# Read in DNM data and annotation

We read DNM data and annotation data using `TADA_A_read_info_by_chunks` and store all the information into `compact_data`, a compact data form benefiting from our categorization trick. This categorization trick greatly reduced the size of the data and facilitates fast parameter inference. 

--mut_files

A vector with names of DNM files. Notice here allelic information needs to be included as our model is allele-aware. Below is an example of the first three rows of one DNM file.


In [12]:
head -n 3 mut_files

chr10	114910882	114910883	G	A
chr9	14150142	14150143	C	T
chr11	66254813	66254814	G	A


--window_file

Must be the same window file used in the mutation rate adjustion step.

--mutrate_scaling_files

A vector with the names of mutation rate scaling files which were generated in the mutation rate adjustion step. The ordering of these files should be consistent with that of the mutation rate files specified by `--mut_files`

--sample_sizes

A vector with the sample sizes of all the studies. The ordering of these numbers should be consistent with the mutation rate files specified by `--mut_files`

--gene_prior_file

A string representing the name of a file with the prior probability of each gene being risk gene. Below is an example of the first three rows of one prior file. 


In [13]:
head -n 3 gene_prior_file

genename	prior
A1BG	0.05
A1CF	0.05


--nonAS_noncoding_annotations

A vector representing the names of non-allele specific annotations. All annotations should be in a 3-column BED format. Each annotation file covers all the bases (or at least all the bases included in the window file) that have that annotation.

--AS_noncoding_annotations

A list representing allele-specific annotations. Each element of the list is a four-element vector of the names of allele-specific annotation files for each annotation. Each allele-specific annotation file of an annotation is in 3-column BED format, covering all the bases (or at least all the bases included in the window file) that have that annotation if mutated to a certain allele. The ordering of these allele-specific annotation files for one annotation strictly follows mutant allele as "A", "C", "G", and "T".

--report_proportion

A number showing the proportion of genes whose DNM and annotation information will be collected and used in relative risk estimation of annotations. All the genes in the `gene_prior_file` will be ranked from high to low based on their prior probabilities. Only the top proportion based on `report_proportion` of all genes will be used. 

--chunk

A number specifying how many partitions will the `window_file` be split into. We do partitions in order to reduce the computation burden by only handling a relatively small set of genomic regions when counting DNM and recording annotations at each base. If only look at top 1000 genes, setting `--chunk_partition_num` to 1 would be usually sufficient.

--node_n

The number of computational nodes that needs to be used.

--mutrate_ref_files

A vector specifying the names of the base-level allele-specific baseline mutation rate files. These files are in the bigWiggle format, storing base-level mutation rates for mutant allele as "A", "C", "G", and "T", sequentially. Users could use their own mutation rate models but need to build their own base-level allele-specific mutation rate files accordingly. 

In [14]:
setwd("/storage11_7T/fuy/TADA-A/annotation")

lst0=c("/storage11_7T/fuy/TADA-A/annotation/PPI/uq.mg.hg19.mapped.bed")

lst=list(
c(
"/storage11_7T/fuy/TADA-A/annotation/MPC_score/v2/v2.nlt2.altA.bed",
"/storage11_7T/fuy/TADA-A/annotation/MPC_score/v2/v2.nlt2.altC.bed",
"/storage11_7T/fuy/TADA-A/annotation/MPC_score/v2/v2.nlt2.altG.bed",
"/storage11_7T/fuy/TADA-A/annotation/MPC_score/v2/v2.nlt2.altT.bed"
))

system.time(compact_data <- TADA_A_read_info_by_chunks(
    
mut_files = "/storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/6788snv.affected.cd.auto.no_pli_rm.allele.bed",
    
window_file = "/storage11_7T/fuy/TADA-A/db/MS_data/windows_partition/no_overlp.17428.gene.syn.new_mutrate.bed",
    
mutrate_scaling_files ="/storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/new_mutrate_no_overlp.1711_aff_syn_scale.txt",
   
sample_sizes = 6430, 
    
gene_prior_file = "/storage11_7T/fuy/TADA-A/db/MS_data/prior/new_uniform_gene_prior.txt",
      
nonAS_noncoding_annotations = NA,

AS_noncoding_annotations = lst,                    

report_proportion = 100/20000,
chunk = 2,
node_n = 1,

    
mutrate_ref_files = c("/storage11_7T/fuy/TADA-A/db/new_mutrate/uq.st.no_overlp.17428.gene.mutrate.A.bw",
                      "/storage11_7T/fuy/TADA-A/db/new_mutrate/uq.st.no_overlp.17428.gene.mutrate.C.bw",
                      "/storage11_7T/fuy/TADA-A/db/new_mutrate/uq.st.no_overlp.17428.gene.mutrate.G.bw",
                      "/storage11_7T/fuy/TADA-A/db/new_mutrate/uq.st.no_overlp.17428.gene.mutrate.T.bw")
))

saveRDS(compact_data,paste0("/storage11_7T/fuy/TADA-A/cell_WES/DNM/rds/",Sys.Date(),"test_compact.rds"))

[1] "Made a tmp folder for tmp files."
[1] "Read in window files."
[1] "Read in mutrate scaling file /storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/new_mutrate_no_overlp.1711_aff_syn_scale.txt"
[1] "Read in gene prior file."
[1] "Chose the top 0.5% genes for parameter estimation."
[1] "Patitioned genomic window to base-level coordinates at Round 1. Time consumed: 1.812s."
[1] "Obtained un-calibrated base-level mutation rate for alt alleles. Time consumed: 40.274.s"
[1] "Added allele specific annotations 1 .Time consumed: 9.741s."
[1] "Begin collapsing data..."
[1] "Read in mutation file /storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/6788snv.affected.cd.auto.no_pli_rm.allele.bed."
[1] "Collapsed data based on annotations for Study 1 .Time consumed: 4.648.s"
[1] "Patitioned genomic window to base-level coordinates at Round 2. Time consumed: 1.662s."
[1] "Obtained un-calibrated base-level mutation rate for alt alleles. Time consumed: 37.566.s"
[1] "Added allele specific annotations 1 .Tim

   user  system elapsed 
146.865   7.175 109.174 

# Feature selection
Estimate the relative risks of each individual annotation supplied to `TADA_A_read_info_by_chunks`, and only use the sinificant ones for the next round of joint estimation. The code below performs relative risk estimation. We estimate the relative risk for each feature sequentially with a fixed prior.


--data

The `base_info` object returned by `TADA_A_read_info_by_chunks`, which stores DNM information, mutation rates, and annotations for each gene in a compact format. 

--selected_annotations

A vector indicating which annotations are used in relative risk estimation. In the feature selection step, we always estimate relative risk for each feature separately, so the vector only has one number specifying which annotation will be used. As you may member, we have multiple annotations specified by `--nonAS_noncoding_annotations` and `AS_noncoding_annotations`. We numbered these annotations from 1 to the total number of annotations. For example, if we have three nonallele-specific annotations `c("nonAS_Annotation_1", "nonAS_Annotation_2", "nonAS_Annotation_3")` specified by `--nonAS_noncoding_annotations`, and two allele-specific annotations `list(c("AS_Annotation_1_alt_A", "AS_Annotation_1_alt_A", "AS_Annotation_1_alt_A", "AS_Annotation_1_alt_A"), c("AS_Annotation_2_alt_A", "AS_Annotation_2_alt_A", "AS_Annotation_2_alt_A", "AS_Annotation_2_alt_A"))` specified by `AS_noncoding_annotations`. Then we have five annotations together, "nonAS_Annotation_1", "nonAS_Annotation_2", "nonAS_Annotation_3", "AS_Annotation_1", and "AS_Annotation_2" will be numbered 1, 2, 3, 4, 5, respectively. If we want to estimate the relative risk of "nonAS_Annotation_3", we set `--selected_annotations` to `c(3)`. If we want to jointly estimate the relative risks of "nonAS_Annotation_3" and "AS_Annotation_1", we set `--selected_annotations` to `c(3,4)`.

--gene_prior_file

A string representing the name of a file with the prior probability of each gene being risk gene. Below is an example of the first three rows of one prior file. In most scenarios, you want to be consistent with `TADA_A_read_info` regarding to the choice of priors.

 --optimization_iteration

The maximum number of iterations when performing optimization. `Optim()` was used for optimization. The search space for when only one parameter is estimated is from -1 to 10.

--mode 

A string that is `"regular"` (default), or `"single_fast"`. `"single_fast"` is used when estimating RR from only one annotation ( when running `TADA_A_read_info_by_chunks`, only one annotation is provided) of lots of genes (e.g., all genes), would be at least 5 times faster.

[output]:
  
Each row of the data.frame is an annotation. Columns are log(Relative risk), 95% confidence interval lower bound of log(RR), and 95% confidence interval upper bound of log(RR).

In [25]:
selected_annotations=c(1)
df=c()
# for(i in selected_annotations){
df2=TADA_A_RR_estimate(data=data$base_info, selected_annotations=i, 
                   gene_prior_file="/storage11_7T/fuy/TADA-A/db/MS_data/prior/new_uniform_gene_prior.txt",
                   optimization_iteration = 2000, mode = "regular",hessian=TRUE)
df=rbind(df,df2)
#     }
# df$annota=paste0("selected annota",c(1))
df

[1] "Read in gene prior file. Time consumed: 0.029s."
[1] "Finished parameter estimation!"


par_estimate,lower,upper
<dbl>,<dbl>,<dbl>
2.338677,1.803951,2.873403


# Jointly estimating the relative risks of annotations
We perform joint estimation for annotations that pass the feature selection step. Example code is shown below. We set `--selected_annotations` to be `c(1,5,8)` as the 1st, 5th and 8th annotations passed the feature selection. 


In [ ]:
TADA_A_RR_estimate(data=data$base_info, selected_annotations=c(1,5,8), 
                   gene_prior_file="/storage11_7T/fuy/TADA-A/db/MS_data/prior/new_uniform_gene_prior.txt",
                   optimization_iteration = 2000, mode = "regular",hessian=TRUE)

# Predict risk genes using SNVs

Use the relative risks of annotations from the last step to identify risk genes. The example code is below. Notice, if previously in relative risk estimation, we only used top genes (e.g., top 1000 genes based on priors), then `compact_data$base_info`, would only contain information for these top 1000 genes. To predict risk genes for all potential genes, we need to first run `TADA_A_read_info_by_chunks` again over all the genes in the `window_file`. 

The difference compared to Step 4 is that 1) now we only take in annotations that have passed the feature selection when running `TADA_A_read_info`; 2) set `--report_proportion` to 1 so all genes in the `window_file` would be included; 3) set `--chunk` to a bigger number such as `20` to avoid memory overflow issue, which would arise when each chunk has to cover too many genomic positions. 

--gama

The `par_estimate` in the last step. We specify the relative risks in the log scale to `gama` for the selected annotations sequentially in a vector. 

**[output]**

List of FDR < 0.1 and FDR_all. `q0` is the posterior probability. We adjusted `q0` using FDR.

In [27]:
library(eTADA)
data=readRDS(paste0("/storage11_7T/fuy/TADA-A/cell_WES/DNM/rds/2022-02-08_new_mutrate_no_overlp_mpc01_520syn_unaff_uni_prior_compact.rds"))

res=TADA_A_get_FDR_SNVs(data=data$base_info,gama=c(2.36,0.1,0.048),
                    selected_annotations=c(1,2),
                    gene_prior_file = "/storage11_7T/fuy/TADA-A/db/MS_data/prior/new_uniform_gene_prior.txt",
                    optimization_iteration=2000)
names(res)
res$FDR_ls_0.1

[1] "FDR_ls_0.1" "FDR_all"

genename,prior,q0,FDR
<chr>,<dbl>,<dbl>,<dbl>
FAM214A,0.9482649,0.05173513,0.05173513


# Estimate frameshift indels mutation rates

-- window_file 

The file with genomic windows. Each line represents one window. The first 4 columns must be chr, start, end, and site_index of genomic windows. The rest of the columns are features that might affect baseline background mutation rates and that need to be adjusted for.


--scaling_file_name 

The name of the file that has the scaling factor for each genomic interval in [window_file]. 1st column is site_index, 2nd column is scaling factor of mutation rate.

--sample_size 

The number of individuals.


--Frameshift_Inframe_ratio 

The ratio of the number of frameshift indels within windows to the number of inframe indels within windows.


--inframe_coverage_file 

Generated by `bedtools coverage -a window_file -b inframe.allele.bed | cut -f 1-13`


--frameshift_coverage_file 

Generated by `bedtools coverage -a window_file -b frameshift.allele.bed | cut -f 1-13`


--pp_file_SNV 

The result of TADA_A_get_FDR_SNVs.R. Posterior Probabilities of all genes after considering all SNV annotations.

In [2]:
setwd("/storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/indel")
library(eTADA)
Frameshift_rate(window_file="/storage11_7T/fuy/TADA-A/db/MS_data/windows_partition/no_overlp.17428.gene.syn.new_mutrate.bed",
                scaling_file_name="/storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/new_mutrate_no_overlp.1711_aff_syn_scale.txt",
                inframe_coverage_file="1base.125inframe.bed.coverage.in_1711aff_syn_scale_17428_wd",
                frameshift_coverage_file="1base.458frameshift.allele.bed.coverage.in_1711aff_syn_scale_17428_wd",      
                sample_size=6430,
                Frameshift_Inframe_ratio=3.5,
                pp_file_SNV="/storage11_7T/fuy/TADA-A/cell_WES/DNM/res/2022-03-10_ASD-df22-estim-pi-all-gene.txt",
                output_file="test.txt")

[1] "The number of inframe shift indels is 125"

Call:
glm(formula = infram_count ~ calib_mutrate_2N, family = poisson(link = "log"), 
    data = df4)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7862  -0.0363  -0.0357  -0.0352   5.3303  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -7.32522    0.09158  -79.99   <2e-16 ***
calib_mutrate_2N  0.16009    0.01086   14.74   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 1830  on 179795  degrees of freedom
Residual deviance: 1771  on 179794  degrees of freedom
AIC: 2019.2

Number of Fisher Scoring iterations: 9



In [4]:
head -n 3 /storage11_7T/fuy/TADA-A/cell_WES/DNM/affected/indel/test.txt

genename	prior	fs_rate_sum	fs_count
A1BG	0.104893836829155	1.35684101625564e-06	0
A1CF	0.103535217290444	2.10221536041985e-06	0


# Predict risk genes after considering INDELs

--gama

Initial value of frameshift log(relative risk).

**[output]**

List of frameshift log(relative risk), genelist of FDR < 0.1, and genelist of FDR_all. `q0` is the posterior probability. We adjusted `q0` using FDR.

In [6]:
df=TADA_A_get_FDR_INDELs(gama=30,sample_size=1000,frameshift_rate_file="test.txt")
names(df)
df$log_frameshift_rr
head(df$FDR_ls_0.1,2)

[1] "log_frameshift_rr" "FDR_ls_0.1"        "FDR_all"

[1] 1.463786

genename,prior,fs_rate_sum,fs_count,post,q0,FDR
<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
SCN2A,1.0000000,4.769147e-06,3,1,2.220446e-16,2.220446e-16
CHD8,0.9999984,6.441148e-06,4,1,5.968040e-09,2.984020e-09
